In [1]:
!pip install -U -q PyDrive

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!mkdir data

In [5]:
file_id = '1KiOvhsdjJqaUCLJa5adZXEtQ_72s8Eb6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/checking-logs.sqlite')

In [6]:
conn = sqlite3.connect('data/checking-logs.sqlite')

In [7]:
commits = pd.io.sql.read_sql(
    """
    select uid, numTrials, timestamp as date
    from checker
    where uid like 'user_%' and status = 'ready' and labname = 'project1'
    """,
    conn, parse_dates=['date']
    )
commits['date'] = commits['date'].dt.floor('d')

In [8]:
conn.close()

In [9]:
commits.head()

,uid,numTrials,date
0,user_4,1,2020-04-17
1,user_4,2,2020-04-17
2,user_4,3,2020-04-17
3,user_4,4,2020-04-17
4,user_4,5,2020-04-17


In [10]:
df = commits.groupby(['date', 'uid']).max()['numTrials'].unstack()
df.index = np.arange(len(df.index))
df.iloc[0] = df.iloc[0].fillna(0)
df = df.fillna(method='pad', axis=0)
df = df.transpose()

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
uid,,,,,,,,,,,,,,,,,,,
user_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,11.0
user_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,21.0,59.0,59.0
user_11,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
user_12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0
user_13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,30.0,30.0,32.0,32.0
user_14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,18.0,25.0,49.0,92.0,92.0,99.0,99.0
user_15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0
user_16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,10.0,10.0
user_17,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,6.0,6.0


In [12]:
initial_data = [go.Scatter(x=np.array([]), y=np.array([]),
                           mode='lines+markers', name=name)
                for name in df.index]

In [13]:
frames = [go.Frame(data=[go.Scatter(
    x=np.arange(i + 1), y=np.array(df.iloc[j, :i + 1]),
    mode='lines+markers', name=df.index[j]
    ) for j in range(len(df.index))]) for i in range(len(df.columns) + 1)]

In [14]:
fig = go.Figure(
    data=initial_data,
    layout={
        'width': 1200,
        'height': 600,
        'title': 'Dynamic of commits per user in project1',
        'xaxis': {'range': (0, len(df.columns) + 1), 'dtick': 2},
        'yaxis': {'range': (0, round(max(df.max()) + 2, -1) + 1)},
        'updatemenus': [{
            'type': 'buttons',
            'buttons': [{'method': 'animate', 'label': 'play', 'args': [None]}]
            }]}, frames=frames)
fig.show()